# dataset path = E:\Mycode\UM_MDSAIA_Assignmentss\CISC7201\final_project\processed_lianjia_data_filtered.csv


In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
import pandas as pd

# Define features and target variable
df = pd.read_csv('E:\\Mycode\\UM_MDSAIA_Assignmentss\\CISC7201\\final_project\\processed_lianjia_data_filtered.csv')
X = df.drop(columns=['price_per_sqm'])
y = df['price_per_sqm']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [11]:
import torch

# 检查是否有可用的GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: cpu


In [14]:
# Define features and target variable for price_per_sqm prediction
X_price = df.drop(columns=['price_per_sqm'])
y_price = df['price_per_sqm']

# Split the data into training and testing sets
X_train_price, X_test_price, y_train_price, y_test_price = train_test_split(X_price, y_price, test_size=0.2, random_state=42)

# Initialize the models
dt_regressor_price = DecisionTreeRegressor(random_state=42)
mlp_regressor_price = MLPRegressor(random_state=42, max_iter=1000)

# Train the Decision Tree Regressor
dt_regressor_price.fit(X_train_price, y_train_price)
y_pred_dt_price = dt_regressor_price.predict(X_test_price)
mse_dt_price = mean_squared_error(y_test_price, y_pred_dt_price)
print(f'Decision Tree Regressor MSE for price_per_sqm: {mse_dt_price}')

# Train the MLP Regressor
mlp_regressor_price.fit(X_train_price, y_train_price)
y_pred_mlp_price = mlp_regressor_price.predict(X_test_price)
mse_mlp_price = mean_squared_error(y_test_price, y_pred_mlp_price)
print(f'MLP Regressor MSE for price_per_sqm: {mse_mlp_price}')

Decision Tree Regressor MSE for price_per_sqm: 3014607.752442997
MLP Regressor MSE for price_per_sqm: 373817.82477807254


In [6]:
import torch
import numpy as np
from sklearn.decomposition import PCA

import torch.nn as nn
import torch.optim as optim

# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# Initialize the MLP model, loss function, and optimizer
mlp_model = MLP(X_train.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(mlp_model.parameters(), lr=0.001)

# Train the MLP model
num_epochs = 1000
for epoch in range(num_epochs):
    mlp_model.train()
    optimizer.zero_grad()
    outputs = mlp_model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

# Evaluate the MLP model
mlp_model.eval()
with torch.no_grad():
    y_pred_mlp_tensor = mlp_model(X_test_tensor)
    mse_mlp = criterion(y_pred_mlp_tensor, y_test_tensor).item()
print(f'Pure MLP Regressor MSE: {mse_mlp}')

# Basic implementation of Random Forest
class RandomForest:
    def __init__(self, n_estimators=100, max_depth=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.trees = [DecisionTreeRegressor(max_depth=self.max_depth) for _ in range(self.n_estimators)]

    def fit(self, X, y):
        for tree in self.trees:
            bootstrap_indices = np.random.choice(np.arange(X.shape[0]), size=X.shape[0], replace=True)
            X_bootstrap = X[bootstrap_indices]
            y_bootstrap = y[bootstrap_indices]
            tree.fit(X_bootstrap, y_bootstrap)

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        return np.mean(tree_preds, axis=0)

# Initialize and train the Random Forest model
rf_model = RandomForest(n_estimators=100, max_depth=10)
rf_model.fit(X_train.values, y_train.values)
# Apply PCA to the dataset
pca = PCA(n_components=0.95)  # Retain 95% of variance
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Convert PCA-transformed data to PyTorch tensors
X_train_pca_tensor = torch.tensor(X_train_pca, dtype=torch.float32)
X_test_pca_tensor = torch.tensor(X_test_pca, dtype=torch.float32)

# Initialize and train the MLP model on PCA-transformed data
mlp_model_pca = MLP(X_train_pca.shape[1])
optimizer_pca = optim.Adam(mlp_model_pca.parameters(), lr=0.001)

for epoch in range(num_epochs):
    mlp_model_pca.train()
    optimizer_pca.zero_grad()
    outputs_pca = mlp_model_pca(X_train_pca_tensor)
    loss_pca = criterion(outputs_pca, y_train_tensor)
    loss_pca.backward()
    optimizer_pca.step()

# Evaluate the MLP model on PCA-transformed data
mlp_model_pca.eval()
with torch.no_grad():
    y_pred_mlp_pca_tensor = mlp_model_pca(X_test_pca_tensor)
    mse_mlp_pca = criterion(y_pred_mlp_pca_tensor, y_test_tensor).item()
print(f'MLP Regressor MSE with PCA: {mse_mlp_pca}')

# Initialize and train the Random Forest model on PCA-transformed data
rf_model_pca = RandomForest(n_estimators=100, max_depth=10)
rf_model_pca.fit(X_train_pca, y_train.values)

# Evaluate the Random Forest model on PCA-transformed data
y_pred_rf_pca = rf_model_pca.predict(X_test_pca)
mse_rf_pca = mean_squared_error(y_test, y_pred_rf_pca)
print(f'Random Forest Regressor MSE with PCA: {mse_rf_pca}')

# Evaluate the Random Forest model
y_pred_rf = rf_model.predict(X_test.values)
mse_rf = mean_squared_error(y_test, y_pred_rf)
print(f'Pure Random Forest Regressor MSE: {mse_rf}')

Pure MLP Regressor MSE: 546370496.0
MLP Regressor MSE with PCA: 1530772864.0
Random Forest Regressor MSE with PCA: 210430252.7343058
Pure Random Forest Regressor MSE: 6927773.97039163


In [ ]:
# Pure Decision Tree Analysis for Prediction
# Initialize the Decision Tree model
dt_regressor = DecisionTreeRegressor(random_state=42)

# Train the Decision Tree model
dt_regressor.fit(X_train, y_train)

# Predict on the test set
y_pred_dt = dt_regressor.predict(X_test)

# Calculate Mean Squared Error
mse_dt = mean_squared_error(y_test, y_pred_dt)
print(f'Mean Squared Error of the pure Decision Tree regression model: {mse_dt}')

# Version with PCA followed by Random Forest
# Perform PCA dimensionality reduction on the dataset
pca = PCA(n_components=0.95)  # Retain 95% of variance
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Initialize the Random Forest model using individual Decision Trees
class RandomForest:
    def __init__(self, n_estimators=100, max_depth=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.trees = [DecisionTreeRegressor(max_depth=self.max_depth) for _ in range(self.n_estimators)]

    def fit(self, X, y):
        for tree in self.trees:
            bootstrap_indices = np.random.choice(np.arange(X.shape[0]), size=X.shape[0], replace=True)
            X_bootstrap = X[bootstrap_indices]
            y_bootstrap = y[bootstrap_indices]
            tree.fit(X_bootstrap, y_bootstrap)

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        return np.mean(tree_preds, axis=0)

# Initialize the Random Forest model
rf_model_pca = RandomForest(n_estimators=100, max_depth=10)

# Train the Random Forest model
rf_model_pca.fit(X_train_pca, y_train.values.ravel())  # Ensure y_train is a 1D array

# Predict on the test set
y_pred_rf_pca = rf_model_pca.predict(X_test_pca)

# Calculate Mean Squared Error
mse_rf_pca = mean_squared_error(y_test, y_pred_rf_pca)
# Initialize and train the pure Decision Tree model on PCA-transformed data
dt_regressor_pca = DecisionTreeRegressor(random_state=42)
dt_regressor_pca.fit(X_train_pca, y_train.values.ravel())

# Predict on the test set
y_pred_dt_pca = dt_regressor_pca.predict(X_test_pca)

# Calculate Mean Squared Error
mse_dt_pca = mean_squared_error(y_test, y_pred_dt_pca)
print(f'Mean Squared Error of the pure Decision Tree regression model after PCA: {mse_dt_pca}')
print(f'Mean Squared Error of the Random Forest regression model after PCA: {mse_rf_pca}')

In [7]:
import torch

import torch.nn as nn
import torch.optim as optim

# 定义MLP模型
class GradualMLP(nn.Module):
    def __init__(self, input_size):
        super(GradualMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 转换数据为PyTorch张量
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# 初始化MLP模型、损失函数和优化器
gradual_mlp_model = GradualMLP(X_train.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(gradual_mlp_model.parameters(), lr=0.001)

# 训练MLP模型
num_epochs = 1000
for epoch in range(num_epochs):
    gradual_mlp_model.train()
    optimizer.zero_grad()
    outputs = gradual_mlp_model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    # 每100个epoch进行一次强化训练
    if (epoch + 1) % 100 == 0:
        with torch.no_grad():
            y_pred_train = gradual_mlp_model(X_train_tensor)
            train_loss = criterion(y_pred_train, y_train_tensor).item()
            print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {train_loss:.4f}')

# 评估MLP模型
gradual_mlp_model.eval()
with torch.no_grad():
    y_pred_gradual_mlp = gradual_mlp_model(X_test_tensor)
    mse_gradual_mlp = criterion(y_pred_gradual_mlp, y_test_tensor).item()
print(f'Gradual MLP Regressor MSE: {mse_gradual_mlp}')

Epoch [100/1000], Loss: 2162355456.0000
Epoch [200/1000], Loss: 1006230784.0000
Epoch [300/1000], Loss: 979214528.0000
Epoch [400/1000], Loss: 960419264.0000
Epoch [500/1000], Loss: 939120064.0000
Epoch [600/1000], Loss: 913023104.0000
Epoch [700/1000], Loss: 878947968.0000
Epoch [800/1000], Loss: 830806400.0000
Epoch [900/1000], Loss: 758280384.0000
Epoch [1000/1000], Loss: 650082304.0000
Gradual MLP Regressor MSE: 629248064.0


In [ ]:
# Apply PCA to the dataset to reduce dimensions from 79 to 54
pca_54 = PCA(n_components=54)
X_train_pca_54 = pca_54.fit_transform(X_train)
X_test_pca_54 = pca_54.transform(X_test)

# Convert PCA-transformed data to PyTorch tensors
X_train_pca_54_tensor = torch.tensor(X_train_pca_54, dtype=torch.float32)
X_test_pca_54_tensor = torch.tensor(X_test_pca_54, dtype=torch.float32)

# Initialize and train the MLP model on PCA-transformed data
mlp_model_pca_54 = MLP(X_train_pca_54.shape[1])
optimizer_pca_54 = optim.Adam(mlp_model_pca_54.parameters(), lr=0.001)

for epoch in range(num_epochs):
    mlp_model_pca_54.train()
    optimizer_pca_54.zero_grad()
    outputs_pca_54 = mlp_model_pca_54(X_train_pca_54_tensor)
    loss_pca_54 = criterion(outputs_pca_54, y_train_tensor)
    loss_pca_54.backward()
    optimizer_pca_54.step()

# Evaluate the MLP model on PCA-transformed data
mlp_model_pca_54.eval()
with torch.no_grad():
    y_pred_mlp_pca_54_tensor = mlp_model_pca_54(X_test_pca_54_tensor)
    mse_mlp_pca_54 = criterion(y_pred_mlp_pca_54_tensor, y_test_tensor).item()
print(f'MLP Regressor MSE with PCA (54 components): {mse_mlp_pca_54}')

In [2]:
import torch

import torch.nn as nn
import torch.optim as optim

# Define the GradualMLP model
class GradualMLP(nn.Module):
    def __init__(self, input_size):
        super(GradualMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# Initialize the GradualMLP model, loss function, and optimizer
gradual_mlp_model = GradualMLP(X_train.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(gradual_mlp_model.parameters(), lr=0.001)

# Train the GradualMLP model
num_epochs = 1000
for epoch in range(num_epochs):
    gradual_mlp_model.train()
    optimizer.zero_grad()
    outputs = gradual_mlp_model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    # Reinforce training every 100 epochs
    if (epoch + 1) % 100 == 0:
        with torch.no_grad():
            y_pred_train = gradual_mlp_model(X_train_tensor)
            train_loss = criterion(y_pred_train, y_train_tensor).item()
            print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {train_loss:.4f}')

# Evaluate the GradualMLP model
gradual_mlp_model.eval()
with torch.no_grad():
    y_pred_gradual_mlp = gradual_mlp_model(X_test_tensor)
    mse_gradual_mlp = criterion(y_pred_gradual_mlp, y_test_tensor).item()
print(f'Gradual MLP Regressor MSE: {mse_gradual_mlp}')

Epoch [100/1000], Loss: 2004777216.0000
Epoch [200/1000], Loss: 993264512.0000
Epoch [300/1000], Loss: 973791296.0000
Epoch [400/1000], Loss: 953398208.0000
Epoch [500/1000], Loss: 929501888.0000
Epoch [600/1000], Loss: 898565056.0000
Epoch [700/1000], Loss: 854019136.0000
Epoch [800/1000], Loss: 783311232.0000
Epoch [900/1000], Loss: 668801024.0000
Epoch [1000/1000], Loss: 512728480.0000
Gradual MLP Regressor MSE: 494984896.0
